In [4]:
from tagger.core.mongo.models.order import Order, OrderCodeMap
from tagger.models import ExtendedOrder


import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [5]:
valid_orders = Order.objects(orderstatus__nin=[
    "CREATED",
    "RECREATED",
    "PAYMENT_PENDING"
]).all()

In [6]:
for o in valid_orders:
    _code = o.code
    code = _code.replace("ORD-", "") if _code[:4] == "ORD-" else None

    if ExtendedOrder.objects.filter(order_id=o.id).count() > 0:
        eo = ExtendedOrder.objects.get(order_id=o.id)
        if eo.code != code and code is not None:
            print(f"Changing extended order {o.id} code from {eo.code} to {code}")
            eo.code = code
            eo.save()
        elif eo.code is None:
            new_code = ExtendedOrder.generate_usable_code()
            print(f"Changing extended order {o.id} code from {eo.code} to {new_code}")
            eo.code = new_code
            eo.save()
    else:
        e = ExtendedOrder.make_from(o, code=code)
        print(f"Order {o.id} has been extended with code {e.code}! (Given code: {code})")

In [7]:
ExtendedOrder.objects.filter(code=None)

<QuerySet []>